## Bibliotecas

In [0]:
import pyspark.sql.functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [0]:
palette = [
    "#7209B7",  # roxo intenso
    "#F72585",  # rosa vibrante
    "#3A0CA3",  # azul profundo
    "#4CC9F0",  # azul claro
    "#B5179E",  # magenta médio (transição suave entre o rosa e o roxo)
    "#4361EE",  # azul mais vivo, faz ponte com o azul claro
    "#4895EF",  # azul intermediário, complementa a sequência
    "#560BAD",  # roxo mais escuro, para profundidade
    "#3F37C9",  # azul escuro vibrante
    "#F77F00"   # laranja vibrante para contraste quente
]
sns.palplot(palette)

In [0]:
plt.style.use('seaborn-v0_8-dark')

In [0]:
base_path = 'figs'

In [0]:
def grafico_chrun_vs_x(data:pd.DataFrame ,x:list ):
    fig = plt.figure(figsize=(8,5))
    ax = sns.barplot(data=data, x=x, y='total', hue='Churn', palette=palette)

    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{int(height)}', xy=(p.get_x() + p.get_width() / 2, height),
                    xytext=(0, 1), textcoords='offset points',
                    ha='center', va='bottom', fontsize=10, color='black')

    plt.title(f'{x} x Churn')
    plt.xlabel(x)
    plt.ylabel("Quantidade de Churns")
    plt.legend(title="Churn", bbox_to_anchor=(1, 1), loc=2)
    
    return fig

## Importando dataframe

In [0]:
%sql

SELECT * FROM telecom.silver.teleco_cleaned
LIMIT 10;

# Analise 1 - Grupos de clientes

vamos verificar a relação dos tipos de cliente com a variável alvo

(gender | SeniorCitizen | Partner | Dependents |)  VS  chrun

## Gender x Chrun

In [0]:
%sql

    SELECT 
    gender,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS churned,
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS not_churned
    FROM telecom.silver.teleco_cleaned
    GROUP BY gender
    ORDER BY gender;

In [0]:
df_churn_by_gender = spark.sql("""
                               
    SELECT 
    gender,
    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY gender
    ORDER BY gender;


"""
).toPandas()

unpivot no dataframe, para conseguirmos extrair o quantidade churn e não churn por gênero.

In [0]:
df_churn_by_gender = df_churn_by_gender.melt(
    id_vars='gender',
    value_vars=['Yes','No'],
    var_name='Churn',value_name='total')

print(df_churn_by_gender)

In [0]:
fig_churn_gen = grafico_chrun_vs_x(df_churn_by_gender,'gender')
fig_churn_gen.savefig(base_path+'/churn_by_gender.png', format = 'png')
plt.close()


In [0]:
%sql
SELECT Churn,
SUM (CASE WHEN SeniorCitizen = 1 THEN 1 ELSE 0 END) AS Idoso,
SUM (CASE WHEN SeniorCitizen = 0 THEN 1 ELSE 0 END) AS NaoIdoso 
FROM telecom.silver.teleco_cleaned
GROUP BY Churn

In [0]:
df_churn_idoso = df_churn_idoso.melt(id_vars='Churn', value_vars=['Idoso','NaoIdoso'], var_name='Idoso', value_name='total')

## SeniorCitizen x Churn

In [0]:
df_churn_by_senior = spark.sql("""
                               
       SELECT SeniorCitizen,
       SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
       SUM(CASE WHEN churn = 'No' THEN 1 ELSE 0 END )AS No
       FROM telecom.silver.teleco_cleaned
       GROUP BY SeniorCitizen
"""
).toPandas()

In [0]:
df_churn_by_senior = df_churn_by_senior.melt(id_vars = 'SeniorCitizen' , value_vars=['Yes','No'], var_name='Churn', value_name='total')


In [0]:
grafico_chrun_vs_x(df_churn_by_senior,'SeniorCitizen')



In [0]:

fig_churn_senior = grafico_chrun_vs_x(df_churn_by_senior,'SeniorCitizen') 
fig_churn_senior.savefig(base_path+'/churn_by_senior.png', format = 'png')
plt.close()


In [0]:
%sql
SELECT 

## Partner x Chrun

In [0]:
%sql

    SELECT Partner,
    SUM (CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM (CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY Partner
    ORDER BY Partner

In [0]:
df_churn_by_partner = spark.sql("""
    SELECT Partner,
    SUM (CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM (CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY Partner
    ORDER BY Partner
""").toPandas()


In [0]:
df_churn_by_partner = df_churn_by_partner.melt(id_vars='Partner', value_vars=['Yes','No'], var_name='Churn', value_name='total')

In [0]:
grafico_chrun_vs_x(df_churn_by_partner,'Partner')

In [0]:
df_churn_by_partner = grafico_chrun_vs_x(df_churn_by_partner,'Partner') 
df_churn_by_partner.savefig(base_path+'/churn_by_partner.png', format = 'png')
plt.close()

## Dependents x Chrun

In [0]:
%sql

    SELECT Dependents,
    SUM (CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM (CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY Dependents
    ORDER BY Dependents

In [0]:
df_churn_by_dep = spark.sql(
"""

    SELECT Dependents,
    SUM (CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM (CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY Dependents
    ORDER BY Dependents


""").toPandas()
df_churn_by_dep

In [0]:
df_churn_by_dep = df_churn_by_dep.melt(id_vars='Dependents', value_vars=['Yes','No'], var_name='Churn', value_name='total')
df_churn_by_dep

In [0]:
grafico_chrun_vs_x(df_churn_by_dep,'Dependents')

In [0]:
fig_churn_dep = grafico_chrun_vs_x(df_churn_by_dep,'Dependents') 
fig_churn_dep.savefig(base_path+'/churn_by_dep.png', format = 'png')
plt.close()

## Contrato


In [0]:
## Distribuição de churn por tipo de contrato

df_contrato = spark.sql("""

    SELECT Contract,
           SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS churned
        --    SUM(CASE WHEN Churn = 'No' THEN 1 ELSE 0 END) AS No
    FROM telecom.silver.teleco_cleaned
    GROUP BY Contract
    ORDER BY Contract



""").toPandas()
df_contrato

In [0]:


# Ordena os contratos do maior para o menor número de churn
df_churn_by_contract = df_contrato.sort_values(by="churned", ascending=False)

plt.figure(figsize=(8, 5))
ax = sns.barplot(data=df_churn_by_contract, x="Contract", y="churned", palette=palette)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)


for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{int(height)}', xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 1), textcoords='offset points',
                ha='center', va='bottom', fontsize=10, color='black')

plt.title("Número de Clientes que Deram Churn por Tipo de Contrato")
plt.xlabel("Tipo de Contrato")
plt.ylabel("Quantidade de Churns")




plt.tight_layout()

plt.savefig(base_path+'/churn_by_contract.png', format = 'png')
plt.close()

plt.show()

# Grupo 2

## Serviços por Consumidor

PhoneService | MultipleLines | InternetService | StreamingTV | StreamingMovies |

### PhoneService

In [0]:
%sql

SELECT PhoneService,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY PhoneService
ORDER BY COUNT(Churn);

In [0]:
df_phone = spark.sql("""
                     
SELECT PhoneService,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY PhoneService
ORDER BY COUNT(Churn);
                     """).toPandas()

In [0]:
df_phone = df_phone.melt(id_vars= 'PhoneService', value_vars= ['Yes','No'], var_name='Churn', value_name='total')

In [0]:
fig_churn_phone = grafico_chrun_vs_x(df_phone,'PhoneService')
fig_churn_phone.savefig(base_path+'/churn_by_phone.png', format = 'png')
plt.close()

### MultipleLines

In [0]:
df_MultipleLines = spark.sql("""
                     
SELECT MultipleLines,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY MultipleLines
ORDER BY COUNT(Churn);
                     """).toPandas()


In [0]:
df_MultipleLines = df_MultipleLines.melt(id_vars= 'MultipleLines', value_vars= ['Yes','No'], var_name='Churn', value_name='total')
fig_churn_MultipleLines = grafico_chrun_vs_x(df_MultipleLines,'MultipleLines')
fig_churn_MultipleLines.savefig(base_path+'/churn_by_MultipleLines.png', format = 'png')
plt.close()

### InternetService

In [0]:
df_InternetService = spark.sql("""
                     
SELECT InternetService,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn = 'No' THEN 1 ELSE 0 END) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY InternetService
ORDER BY COUNT(Churn);
                     """).toPandas()

In [0]:
df_InternetService

In [0]:
df_InternetService= df_InternetService.melt(id_vars= 'InternetService', value_vars= ['Yes','No'], var_name='Churn', value_name='total')

In [0]:
fig_chrun_internet = grafico_chrun_vs_x(df_InternetService,'InternetService')
fig_chrun_internet.savefig(base_path+'/churn_by_internet.png', format = 'png')
plt.close()

### StreamingTV

In [0]:
%sql 
SELECT StreamingTV,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn =  'No' THEN 1 ELSE 0 END) AS No
from telecom.silver.teleco_cleaned
GROUP BY StreamingTV
ORDER BY COUNT(Churn);

In [0]:
df_streamingtv = spark.sql("""
                         
     SELECT StreamingTV,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn =  'No' THEN 1 ELSE 0 END) AS No
from telecom.silver.teleco_cleaned
GROUP BY StreamingTV
ORDER BY COUNT(Churn);

""").toPandas()

In [0]:
df_streamingtv = df_streamingtv.melt(id_vars= 'StreamingTV', value_vars= ['Yes','No'], var_name='Churn', value_name='total')

fig_churn_streaming = grafico_chrun_vs_x(df_streamingtv,'StreamingTV')
fig_churn_streaming.savefig(base_path+'/churn_by_streamingtv.png', format = 'png')
plt.close()

### StreamingMovies

In [0]:
df_StreamingMovies= spark.sql("""
                         
    SELECT StreamingMovies,
    SUM ( CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN churn =  'No' THEN 1 ELSE 0 END) AS No
from telecom.silver.teleco_cleaned
GROUP BY StreamingMovies
ORDER BY COUNT(Churn);

""").toPandas()

In [0]:
df_StreamingMovies = df_StreamingMovies.melt(id_vars= 'StreamingMovies', value_vars= ['Yes','No'], var_name='Churn', value_name='total')
fig_churn_streaming_movies = grafico_chrun_vs_x(df_StreamingMovies,'StreamingMovies')
fig_churn_streaming_movies.savefig(base_path+'/churn_by_streaming_movies.png', format = 'png')
plt.close()

# Grupo 3 - Suporte e segurança


OnlineSecurity | OnlineBackup | DeviceProtection | TechSupport |

### OlineSecurity

In [0]:
df_security = spark.sql("""
                        
SELECT OnlineSecurity,
  SUM( CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
  SUM( CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY OnlineSecurity

                        """).toPandas()
                        
df_security = df_security.melt(id_vars= 'OnlineSecurity', value_vars= ['Yes','No'], var_name='Churn', value_name='total')
fig_churn_security = grafico_chrun_vs_x(df_security,'OnlineSecurity')
fig_churn_security.savefig(base_path+'/churn_by_security.png', format = 'png')
plt.close()




### OnlineBackup

In [0]:
df_churn_by_onlinebk = spark.sql("""
                                 
SELECT OnlineBackup,
    SUM ( CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
from telecom.silver.teleco_cleaned
GROUP BY OnlineBackup
                                 """).toPandas()
df_churn_by_onlinebk = df_churn_by_onlinebk.melt(id_vars= 'OnlineBackup', value_vars= ['Yes','No'], var_name='Churn', value_name='total')
fig_churn_onlinebk = grafico_chrun_vs_x(df_churn_by_onlinebk,'OnlineBackup')
fig_churn_onlinebk.savefig(base_path+'/churn_by_onlinebk.png', format = 'png')
plt.close()


### DeviceProtection


In [0]:
df_deviceprotection = spark.sql("""
SELECT DeviceProtection,
    SUM ( CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
from telecom.silver.teleco_cleaned
GROUP BY DeviceProtection
""").toPandas()

df_deviceprotection = df_deviceprotection.melt(id_vars= 'DeviceProtection', value_vars= ['Yes','No'], var_name='Churn', value_name='total')

fig_churn_deviceprotection = grafico_chrun_vs_x(df_deviceprotection,'DeviceProtection')
fig_churn_deviceprotection.savefig(base_path+'/churn_by_deviceprotection.png', format = 'png')
plt.close()


### TechSupport


In [0]:
df_techsupport = spark.sql("""
SELECT TechSupport,
    SUM ( CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
    SUM ( CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
from telecom.silver.teleco_cleaned
GROUP BY TechSupport
""").toPandas()

df_techsupport = df_techsupport.melt(id_vars= 'TechSupport', value_vars= ['Yes','No'], var_name='Churn', value_name='total')

fig_churn_techsupport = grafico_chrun_vs_x(df_techsupport,'TechSupport')
fig_churn_techsupport.savefig(base_path+'/churn_by_techsupport.png', format = 'png')
plt.close()

In [0]:
plt.pie(df_contrato['churned'], labels=df_contrato['Contract'], autopct='%1.1f%%',colors=palette,wedgeprops={'width': 0.1})
# plt.legend('upper right')
plt.title('Percent Churn by Contract')
plt.savefig(base_path+'/percent_churn_by_contract.png', format = 'png')

## Features Categoricas x Chrun Cases

In [0]:
%sql

SELECT PaymentMethod,
 SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Churned
FROM telecom.silver.teleco_cleaned
GROUP BY PaymentMethod
ORDER BY Churned DESC

Databricks visualization. Run in Databricks to view.

In [0]:
pd_paymethods = spark.sql("""
                          
SELECT PaymentMethod,
 SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Churned
FROM telecom.silver.teleco_cleaned
GROUP BY PaymentMethod
ORDER BY Churned DESC

    """).toPandas()

In [0]:
plt.pie(pd_paymethods['Churned'], labels=pd_paymethods['PaymentMethod'], autopct='%1.1f%%',colors=palette, wedgeprops={'width': 0.1})
plt.title('Percent Churn by Payment Method')
plt.savefig(base_path+'/percent_churn_by_paymentmethod.png', format = 'png')

# Features Numericas

### Distribuição das features numericas

In [0]:
pd_tenure = spark.sql("""

    SELECT tenure,
    SUM (CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS Churned
    from telecom.silver.teleco_cleaned
    GROUP BY tenure
    ORDER BY Churned

""").toPandas()
pd_tenure

## Analise de Fidelização (Tenure)

In [0]:
df_tenure  = spark.sql("""


SELECT tenure
    COUNT(CASE WHEN Churn = 'Yes' THEN 1 END) Churn
    COUNT(*) AS NotChurned
FROM telecom.silver.teleco_cleaned

""").toPandas()

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT tenure,
    COUNT(CASE WHEN Churn = 'Yes' THEN 1 END) Churn,
    COUNT(*) AS NotChurned
FROM telecom.silver.teleco_cleaned
GROUP BY tenure

In [0]:
%sql
SELECT 
    MonthlyCharges,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churn
FROM telecom.silver.teleco_cleaned
GROUP BY MonthlyCharges
ORDER BY Churn DESC;

In [0]:
plt.figure(figsize=(8,5))
sns.histplot(df_tenure['tenure'],kde=True,bins = 50,color=palette[2])
plt.ylabel('Density')
plt.xlabel('Tenure')
plt.xticks(np.arange(0,df_tenure['tenure'].max()+1,3))
plt.title('Tenure Distribution')
plt.savefig(base_path+'/tenure_distribution.png')

In [0]:
%sql
SELECT tenure,
SUM (CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
SUM (CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY tenure
ORDER BY tenure ASC;

In [0]:
df_churn_tenure = spark.sql("""

SELECT tenure,
SUM (CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Yes,
SUM (CASE WHEN Churn = 'No' THEN 1 ELSE 0 END ) AS No
FROM telecom.silver.teleco_cleaned
GROUP BY tenure
ORDER BY tenure ASC;
                          
                            """).toPandas()

Propoção de Churn = 

In [0]:
df_churn_tenure['tenure_bin'] = pd.cut(df_churn_tenure['tenure'],bins=[0,6,12,24,36,48,60,72])
# Agrupar os dados por bins e somar as quantidades
df_grouped = df_churn_tenure.groupby('tenure_bin')[['Yes', 'No']].sum().reset_index()
# Calcular a proporção de churn
df_grouped['ChurnRate'] = df_grouped['Yes'] / (df_grouped['Yes'] + df_grouped['No'])

In [0]:
df_grouped

In [0]:
# Plotar o gráfico de barras
plt.figure(figsize=(8,6))
ax = sns.barplot(data=df_grouped, x='tenure_bin', y='ChurnRate', color=palette[5])

# Adicionar os valores acima das barras
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height:.2%}', xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points',
                ha='center', va='bottom', fontsize=10, color='black')

plt.title('Proporção de Churn por Tempo de Contrato (Tenure)')
plt.ylabel('Proporção de Churn')
plt.xlabel('Tempo de Contrato (meses)')
plt.ylim(0, 1)  # Porque a proporção varia entre 0 e 1
plt.show()

In [0]:
%sql
SELECT 
      COUNT(CASE WHEN Churn = 'Yes' THEN 1  END) AS Churned,
      COUNT(CASE WHEN Churn = 'No' THEN 1  END) AS NotChurned,
      ROUND((COUNT(CASE WHEN Churn = 'Yes' THEN 1  END) * 1.0) / COUNT(*),3) AS ChurnRate,
      CASE
            WHEN tenure >= 0 AND tenure < 6 THEN '(0-6]'
            WHEN tenure >= 6 AND tenure < 12 THEN '(6-12]'
            WHEN tenure >= 12 AND tenure < 18 THEN '(12-18]'
            WHEN tenure >= 18 AND tenure < 24 THEN '(18-24]'
            WHEN tenure >= 24 AND tenure < 36 THEN '(24-36]'
            WHEN tenure >= 36 AND tenure < 48 THEN '(36-48]'
            WHEN tenure >= 48 AND tenure < 60 THEN '(48-60]'
            WHEN tenure >= 60 AND tenure <= 72 THEN '(60-72)'
      END AS tenure_bin
FROM telecom.silver.teleco_cleaned
GROUP BY tenure_bin
ORDER BY ChurnRate DESC;

In [0]:
df_churnrate_tenure = spark.sql("""

SELECT 
      COUNT(CASE WHEN Churn = 'Yes' THEN 1  END) AS Churned,
      COUNT(CASE WHEN Churn = 'No' THEN 1  END) AS NotChurned,
      ROUND((COUNT(CASE WHEN Churn = 'Yes' THEN 1  END) * 1.0) / COUNT(*),3) AS ChurnRate,
      CASE
            WHEN tenure >= 0 AND tenure < 6 THEN '(0-6]'
            WHEN tenure >= 6 AND tenure < 12 THEN '(6-12]'
            WHEN tenure >= 12 AND tenure < 18 THEN '(12-18]'
            WHEN tenure >= 18 AND tenure < 24 THEN '(18-24]'
            WHEN tenure >= 24 AND tenure < 36 THEN '(24-36]'
            WHEN tenure >= 36 AND tenure < 48 THEN '(36-48]'
            WHEN tenure >= 48 AND tenure < 60 THEN '(48-60]'
            WHEN tenure >= 60 AND tenure <= 72 THEN '(60-72)'
      END AS tenure_bin
FROM telecom.silver.teleco_cleaned
GROUP BY tenure_bin
ORDER BY ChurnRate DESC;

""").toPandas()

In [0]:
df_churnrate_tenure

In [0]:
# Plotar o gráfico de barras
plt.figure(figsize=(8,6))
ax = sns.barplot(data=df_churnrate_tenure, x='tenure_bin', y='ChurnRate', color=palette[2])

# Adicionar os valores acima das barras
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{height:.2%}', xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 3), textcoords='offset points',
                ha='center', va='bottom', fontsize=10, color='black')

plt.title('Churn Ratio by Tenure')
plt.ylabel('Churn Ratio')
plt.xlabel('Tenure')
plt.ylim(0, 1)  # Porque a proporção varia entre 0 e 1
plt.savefig(base_path+'/prop_churn_tenure.png', format = 'png')
plt.show()


## Payment Anali


In [0]:
%sql
SELECT MonthlyCharges,
--  any_value(Churn) as Churn
COUNT(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churn
FROM telecom.silver.teleco_cleaned
GROUP BY MonthlyCharges
ORDER BY MonthlyCharges DESC


In [0]:
df_totalcharge = spark.sql(
    """
SELECT TotalCharges,
--  any_value(Churn) as Churn
COUNT(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churn
FROM telecom.silver.teleco_cleaned
GROUP BY TotalCharges
ORDER BY TotalCharges DESC
    """
).toPandas()

In [0]:
fig, ax = plt.subplots(figsize=(8,5))
sns.histplot(df_totalcharge['TotalCharges'],bins = 30,color=palette[2],kde=True)
plt.xticks(np.arange(0, max(df_totalcharge['TotalCharges'].values+1), df_totalcharge['TotalCharges'].values.max()/10))
plt.ylabel('Density')
plt.xlabel('TotalCharges')
plt.savefig(base_path+'/dist_totalcharges.png', format = 'png')

In [0]:
df_totalcharge = spark.sql("""
    
SELECT
    CASE
        WHEN TotalCharges >= 0 AND TotalCharges < 1000 THEN '0-1000'
        WHEN TotalCharges >= 1000 AND TotalCharges < 2000 THEN '1000-2000'
        WHEN TotalCharges >= 2000 AND TotalCharges < 3000 THEN '2000-3000'
        WHEN TotalCharges >= 3000 AND TotalCharges < 4000 THEN '3000-4000'
        WHEN TotalCharges >= 4000 THEN '4000+'
        ELSE 'Unknown'
    END AS TotalCharges_bin,
    COUNT(CASE WHEN Churn = 'Yes' THEN 1 END) AS Churn,
    COUNT(CASE WHEN Churn = 'No' THEN 1 END) AS NoChurn
FROM telecom.silver.teleco_cleaned
WHERE TotalCharges IS NOT NULL
GROUP BY TotalCharges_bin
ORDER BY TotalCharges_bin ASC;


                           """).toPandas()

In [0]:
# Supondo que o seu df está assim:
# TotalCharges_bin | Churn | NoChurn

df_melted = df_totalcharge.melt(id_vars='TotalCharges_bin', 
                                value_vars=['Churn', 'NoChurn'], 
                                var_name='ChurnStatus', 
                                value_name='Count')


In [0]:
plt.figure(figsize=(10,6))
ax = sns.barplot(data=df_melted, x='TotalCharges_bin', y='Count', hue='ChurnStatus', palette=[palette[1], palette[2]])

# Adicionar os valores nas barras
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{int(height)}',
                (p.get_x() + p.get_width() / 2, height),
                ha='center', va='bottom', fontsize=10, color='black')

plt.title('Distribuição de Churn por Faixa de TotalCharges')
plt.xlabel('Faixa de TotalCharges')
plt.ylabel('Número de Clientes')
plt.legend(title='Status')
plt.savefig(base_path + '/prop_churn_totalcharges.png', format='png')
plt.show()



In [0]:
df_totalcharge.head()

In [0]:
df_totalcharge = spark.sql("""

SELECT
    CASE
        WHEN TotalCharges >= 0 AND TotalCharges < 1000 THEN '0-1000'
        WHEN TotalCharges >= 1000 AND TotalCharges < 2000 THEN '1000-2000'
        WHEN TotalCharges >= 2000 AND TotalCharges < 3000 THEN '2000-3000'
        WHEN TotalCharges >= 3000 AND TotalCharges < 4000 THEN '3000-4000'
        WHEN TotalCharges >= 4000 THEN '4000+'
        ELSE 'Unknown'
    END AS CobrancaTotal,
    
    COUNT(*) AS TotalClientes,
    AVG(tenure) AS Media_Tempo_de_Contrato,
    STDDEV(tenure) AS Desvio_Tempo_de_Contrato,
    AVG(MonthlyCharges) AS Media_Pagamento_Mensal,
    STDDEV(MonthlyCharges) AS Desvio_Pagamento_Mensal,
    
    COUNT(CASE WHEN Churn = 'Yes' THEN 1 END) AS TotalChurn,
    (COUNT(CASE WHEN Churn = 'Yes' THEN 1 END) * 1.0) / COUNT(*) AS ProporcaoChurn

FROM telecom.silver.teleco_cleaned
WHERE TotalCharges IS NOT NULL
GROUP BY CobrancaTotal
ORDER BY CobrancaTotal ASC;
"""
).toPandas()

In [0]:
df_totalcharge

In [0]:
fig, ax = plt.subplots(figsize=(10,6))
width = 0.35  # Largura das barras

x = np.arange(len(df_totalcharge['CobrancaTotal']))

bar1 = ax.bar(x - width/2, df_totalcharge['Media_Tempo_de_Contrato'], width, label='Media_Tempo_de_Contrato', color=palette[1])
bar2 = ax.bar(x + width/2, df_totalcharge['Media_Pagamento_Mensal'], width, label='Media_Pagamento_Mensal', color=palette[5])

# Adicionar valores nas barras
for bars in [bar1, bar2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}', (bar.get_x() + bar.get_width() / 2, height),
                    ha='center', va='bottom', fontsize=10)

ax.set_xticks(x)
ax.set_xticklabels(df_totalcharge['CobrancaTotal'])
ax.set_title('Média de Tempo de contrato e pagamento mensal por Faixa de Cobrança Total')
ax.set_xlabel('Faixa de Cobrança Total')
ax.set_ylabel('Média')
ax.legend()
plt.savefig(base_path +'/media_tenure_monthlycharges.png', format='png')
plt.show()
plt.close()


In [0]:
df_monthcharge = spark.sql(
"""
SELECT MonthlyCharges,
COUNT (CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churn
FROM telecom.silver.teleco_cleaned
GROUP BY MonthlyCharges
ORDER BY Churn DESC
"""

).toPandas()

In [0]:
plt.figure(figsize=(8,6))
sns.histplot(df_monthcharge, x='MonthlyCharges', bins=30, kde=True, color=palette[2])
plt.yticks(np.arange(0, 100, 10))
plt.title('Distribuição de MonthlyCharges')
plt.xlabel('MonthlyCharges')
plt.ylabel('Frequência')
plt.savefig(base_path + '/monthlycharges.png', format='png')
plt.show()

In [0]:
%sql
SELECT Churn,
      COUNT(*) AS Total
FROM telecom.silver.teleco_cleaned
GROUP BY Churn





In [0]:

def func(pct, allvals):
    absolute = int(pct/100.*sum(allvals))
    return f'{pct:.1f}%\n({absolute} clientes)'

df_churn = spark.sql("""
SELECT 
    Churn,
    COUNT(*) AS TotalClientes,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS Percentual
FROM telecom.silver.teleco_cleaned
GROUP BY Churn
ORDER BY Churn;                  """).toPandas()


plt.figure(figsize=(8,6))

plt.pie(
    df_churn['TotalClientes'], 
    labels=df_churn['Churn'], 
    autopct=lambda pct: func(pct, df_churn['TotalClientes']),
    colors=palette, 
    wedgeprops={'width': 0.1}  
)

plt.title('Percentual e Quantidade de Churn na Base de Dados')
plt.savefig(base_path + '/percent_churn_total.png', format='png')
plt.show()


In [0]:
df_techsupport = df_techsupport.melt(id_vars= 'TechSupport', value_vars= ['Yes','No'], var_name='Churn', value_name='total')